# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/model/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

from pipeline_config import *
from reversed_hgru_data_preprocess.preprocess_config import *

import os

In [3]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [4]:
hgru_test_preds_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/test_predictions.pickle'

# Read Results Data

In [5]:
with open(test_predictions_path_with_hgru, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(hgru_test_preds_path, 'rb') as f:
    hgru_prediction_dic = pickle.load(f)

with open(canada_path, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [6]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5247/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5247/1314998454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5247/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [7]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [8]:
len(all_data_test_dict.keys())

293

In [9]:
categories_per_indent_dict ={key: categories_per_indent_dict[key] for key in categories_per_indent_dict.keys() if key in [0,1,2,3]}
cat_ids = list(categories_per_indent_dict.values())

flat_cat_id = [item for sublist in cat_ids for item in sublist]

all_categories = []
for cat_id in flat_cat_id:
    all_categories.append(category_id_to_name_dict[cat_id])

#all_categories = list(all_data_test_dict.keys())

In [10]:
all_categories = list(all_data_test_dict.keys())

In [11]:
len(all_categories)

293

In [12]:
def plot_results(all_data_dict, categories):
    category_samples = random.sample(categories, 20)+['All-items']
    #category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [13]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'RMSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [14]:
avg_rmse(all_data_test_dict)

RMSE:  1.681200855223522
RMSE std:  1.4787869505613453
interval: [0.20241390466217668, 3.1599878057848674]


(1.681200855223522, 1.4787869505613453)

In [15]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [16]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 147


137.77755665665754

In [17]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions['Indent'].unique().item())

        if key =='All-items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')


    return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [18]:
avg_r_squared(all_data_test_dict)

Average R Squared:  0.16532051289295094
R Squared std:  0.3771181150229762
interval: [-0.21179760213002527, 0.5424386279159271]
--------------------------------------------------------
Headline R2: 0.23646404167320512
R2 list percentiles:
[10: -0.13544429646062917, 25: 8.812804924507311e-05, 50: 0.18561928438976427, 75: 0.34719297508525915, 90: 0.5491536899024597]
num of neg/zero R^2: 73
num of postive R^2: 220
num of high R^2: 39


(0.23646404167320512,
 [4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  6.0,
  6.0,
  6.0],
 [4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.

In [19]:
len(all_categories)

293

In [20]:
plot_results(all_data_test_dict, all_categories)

Category is: Electricity
RMSE is: 1.1653658014619448


Category is: Infant formula
RMSE is: 0.9963862514410329


Category is: Recreational services
RMSE is: 0.850006789269291


Category is: Apples
RMSE is: 2.746365466947366


Category is: Other household appliances
RMSE is: 2.218310564780601


...

Category is: Toiletry items and cosmetics
RMSE is: 0.9186283547638172


Category is: Communications
RMSE is: 1.6852148368091795


Category is: All other miscellaneous food preparations
RMSE is: 1.3391717514091228


Category is: Cellular services
RMSE is: 3.914049442886889


Category is: All-items
RMSE is: 0.3982382562288441


In [21]:
def create_rmse_table(all_data_test_dict):
    categories = []
    indents = []
    mses = []
    rmses = []
    for key in all_categories:
        categories.append(key)

        df_predictions = all_data_test_dict[key]
        indents.append(df_predictions.Indent.unique().item())
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)

        mses.append(mse)
        rmses.append(np.sqrt(mse))
    
    series1 = pd.DataFrame(categories).reset_index(drop=True).rename(columns={0:'Category'})
    series2 = pd.DataFrame(indents).reset_index(drop=True).rename(columns={0:'Indent'})
    series3 = pd.DataFrame(mses).reset_index(drop=True).rename(columns={0:'MSE'})
    series4 = pd.DataFrame(rmses).reset_index(drop=True).rename(columns={0:'RMSE'})

    df1 = pd.concat([series1, series2], axis = 1)
    df2 = pd.concat([df1, series3], axis = 1)
    df3 = pd.concat([df2, series4], axis = 1)

    return df3


In [22]:
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [23]:
path = os.getcwd()

desired_path = path + '/rmse_per_category_canada.csv'

rmse_per_cat = create_rmse_table(all_data_test_dict)
rmse_per_cat.to_csv(desired_path, index=False)

In [24]:
rmse_per_cat.sort_values('Indent')

,Category,Indent,MSE,RMSE
292,All-items,0.0,0.158594,0.398238
170,"Alcoholic beverages, tobacco products and recr...",1.0,0.110032,0.331711
171,"All-items excluding alcoholic beverages, tobac...",1.0,0.201755,0.449172
172,All-items excluding energy,1.0,0.088415,0.297347
173,All-items excluding food,1.0,0.223358,0.472607
...,...,...,...,...
268,Bacon,6.0,9.695725,3.113796
267,All other miscellaneous food preparations,6.0,1.793381,1.339172
291,Whole milk,6.0,1.644580,1.282412
279,Instant and other coffee,6.0,5.298955,2.301946


In [25]:
def get_df_with_predictions_with_hgru(prediction_dic: dict, hgru_pred_dic: dict, dict_of_categories_df:dict) -> dict:
    bidirect_hgru_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        hgru_predictions = hgru_pred_dic[key]
        hgru_prediction_df =  pd.DataFrame(hgru_predictions.flatten().detach().numpy())

        prediction_df.rename(columns = {0: 'Bidirectional_Prediction'}, inplace=True)
        hgru_prediction_df.rename(columns = {0: 'HGRU_Prediction'}, inplace=True)

        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)

        bidirect_hgru_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df, hgru_prediction_df], axis=1)

    return bidirect_hgru_dict

bidirect_hgru_dict=get_df_with_predictions_with_hgru(prediction_dic, hgru_prediction_dic, test_dict)

In [26]:
def plot_results_bidirect_hgru(bidirect_hgru_dict, categories):
    #category_samples = random.sample(categories, 20)+['All items']
    category_samples = ['All-items'] + categories
    for category in category_samples:
        category_df = bidirect_hgru_dict[category]
        y_pred_bidirect = category_df['Bidirectional_Prediction'].values
        y_pred_hgru = category_df['HGRU_Prediction'].values
        y_actual = category_df['Actual'].values
        bidirect_mse = mean_squared_error(y_pred_bidirect, y_actual)
        hgru_mse = mean_squared_error(y_pred_hgru, y_actual)
        print(f'Category is: {category}')
        print(f'Bidirectional RMSE is: {np.sqrt(bidirect_mse)}')
        print(f'HGRU RMSE is: {np.sqrt(hgru_mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Bidirectional_Prediction", "HGRU_Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [27]:
categories_with_highest_diff = ['Air transportation',
'Ham excluding deli ham',
'Lettuce',
'Books and reading material (excluding textbooks',
'Inter-city transportation',
'Other fresh or frozen poultry',
'Rental of passenger vehicles',
'Traveller accommodation',
'Clothing and footwear',
"Children's clothing",
'Reading material (excluding textbooks',
'Frozen fruit',
"Children's footwear (excluding athletic",
'Apples',
'Fuel oil and other fuels',
'Digital computing equipment and devices',
'Dried and dehydrated fruit',
'Jewellery',
'Other fresh fruit',
'Fresh vegetables',
'Clothing',
'Purchase of digital media',
'Travel tours',
'All other passenger vehicle operating expenses',
'Vegetables and vegetable preparations',
'Other edible fats and oils',
'Other fresh or frozen meat (excluding poultry',
"Women's footwear (excluding athletic",
'Tomatoes',
'Child care services',
'Oranges',
'Telephone services',
'Audio equipment',
'Sporting and exercise equipment',
'Fresh fruit and vegetables',
'Athletic footwear',
'Canned vegetables and other vegetable preparations',
'Footwear',
"Women's clothing",
'Clothing material, notions and services',
'Fresh fruit',
"Men's footwear (excluding athletic",
'Multipurpose digital devices',
'Plastic and aluminum foil supplies',
'Canned fruit',
'Flour and flour-based mixes',
"Men's clothing",
'Public transportation',
'Soup',
'Jam, jelly and other preserves',
'Fresh or frozen beef',
'Child care and housekeeping services',
'Other recreational equipment',
'Other household appliances',
'Potato chips and other snack products n.e.c',
'Processed cheese',
'Personal soap',
'Other cultural and recreational services',
'Fresh or frozen chicken',
'Travel services',
'Photographic equipment and supplies',
'Potatoes',
'Margarine',
'Toys, games (excluding video games) and hobby supplies',
'Toiletry items and cosmetics',
'Gasoline'
                                ]

In [28]:
plot_results_bidirect_hgru(bidirect_hgru_dict, categories_with_highest_diff)

Category is: All-items
Bidirectional RMSE is: 0.3982382562288441
HGRU RMSE is: 0.4205843017294529


Category is: Air transportation
Bidirectional RMSE is: 9.277487150020553
HGRU RMSE is: 12.468798779016378


Category is: Ham excluding deli ham
Bidirectional RMSE is: 3.289361179737147
HGRU RMSE is: 6.402276464942203


Category is: Lettuce
Bidirectional RMSE is: 6.0954601906717345
HGRU RMSE is: 8.58671947435026


Category is: Books and reading material (excluding textbooks
Bidirectional RMSE is: 1.624022325670805
HGRU RMSE is: 4.044523435508292


...

Category is: Potatoes
Bidirectional RMSE is: 3.8348101587306
HGRU RMSE is: 4.339076765594126


Category is: Margarine
Bidirectional RMSE is: 3.1412842486619605
HGRU RMSE is: 3.643492907461552


Category is: Toys, games (excluding video games) and hobby supplies
Bidirectional RMSE is: 1.1327414902408772
HGRU RMSE is: 1.6227336253627944


Category is: Toiletry items and cosmetics
Bidirectional RMSE is: 0.9186283547638172
HGRU RMSE is: 1.4036871377390263


Category is: Gasoline
Bidirectional RMSE is: 5.825006467386685
HGRU RMSE is: 6.30851554829228
